# Running on Apple's Neural Engines

This notebook is an experiment in converting my Torch models into something that
can run on the integrated Neural Engines on my Mac Mini.

This process is pretty well documented, e.g., [here in the coremltools docs](https://coremltools.readme.io/v6.3/docs/pytorch-conversion), but maybe I can add a little context.

You need to install coremltools for this:

```
uv pip install --group apple
```

In [ ]:
import torch
import import_ipynb
import training  # type: ignore
import openai  # type: ignore
import gpt  # type: ignore
import chat  # type: ignore
import coremltools as ct
import numpy as np

scikit-learn version 1.7.0 is not supported. Minimum required version: 0.17. Maximum required version: 1.5.1. Disabling scikit-learn conversion API.
TensorFlow version 2.19.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.
Torch version 2.7.1 has not been tested with coremltools. You may run into unexpected errors. Torch 2.5.0 is the most recent version that has been tested.


## Construct and load the model

You should make sure `model` is an instance of the actual model you want to
export. Make sure you know the context length, you'll need it in the next step.

For now, I'll construct the model, load the weights, and do a quick test to make sure
it works.

(Note: I'm not distributing the `.pth` file for this one, so don't expect this to run on a fresh clone. You'll need
your own model, which you can create in [chat.ipynb](./chat.ipynb).)

In [ ]:
model = gpt.GPTModel(openai.GPT_CONFIG_355M)
training.load(
    model, optimizer=None, name="355M-alpaca", base_path="../models", device="cpu"
)
model.to(gpt.get_device())
chat.instruct(model, "Tell the user that it was successful.")

It was successful!


## Construct a dummy input vector

Here we need an input vector that we'll run through the model using `torch.jit.trace`. The vector should be the maximum dimensions that you want the model to support, so in this case $1\times\text{context\_length}$, or `[1, 1024]`.

In [4]:
device = gpt.get_device()
dummy = torch.ones(1, 1024).long().to(device)

## Create a traced model

> Using torch.jit.trace and torch.jit.trace_module, you can turn an existing module or Python function into a TorchScript ScriptFunction or ScriptModule. You must provide example inputs, and we run the function, recording the operations performed on all the tensors.

—[torch.jit.trace docs](https://docs.pytorch.org/docs/stable/generated/torch.jit.trace.html#torch-jit-trace)

This step transforms the model into a form that can be easily exported. There are tons of caveats here, and not every model would work. As far as I can tell, this one does.

In [44]:
model.eval()
traced_model = torch.jit.trace(model, dummy)

<string>:7: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.


## Convert the traced model to a CoreML package

The final step uses `coremltools` to convert the traced model into an
`mlpackage`, which is a bundle of weights, JSON, etc., that you can import into
Xcode to use in a Swift app.

Note that it the output file should start with a capital letter. It sounds weird,
but Xcode will automatically generate a class based on the name and classes should
start with capitals.

In [45]:
cml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[
        ct.TensorType(shape=[1, ct.RangeDim(1, 1024)], dtype=np.float32, name="in_idx")
    ],
)
cml_model.save("Chatmodel.mlpackage")

Running MIL default pipeline:   9%|▉         | 8/89 [00:00<00:02, 35.37 passes/s]/Users/justin/src/llm-scratch/.venv/lib/python3.11/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:273: UserWarning: Output, '2489', of the source model, has been renamed to 'var_2489' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL default pipeline:  62%|██████▏   | 55/89 [00:02<00:01, 31.76 passes/s]/Users/justin/src/llm-scratch/.venv/lib/python3.11/site-packages/coremltools/converters/mil/mil/ops/defs/iOS15/elementwise_unary.py:889: RuntimeWarning: overflow encountered in cast
  return input_var.val.astype(dtype=string_to_nptype(dtype_val))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 61.15 passes/s]


## Use it in your app

If you want to see an example of this, check out [CrapGPT](https://github.com/juxtin/crapgpt).